In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from transformers import BertForSequenceClassification,BertModel
from imblearn.over_sampling import SMOTE
from utils import CLSdata
import numpy as np
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Subset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = "cpu"


These two codeblocks was used to debug the dataloader.

In [2]:




# csv_file = 'src/transformed/data.csv'
# npy_dir = 'src/embedded_ef/'

# dataset = CLSdata(csv_file=csv_file, npy_dir=npy_dir)
# # dataset = SimpleCLSdata(npy_dir=npy_dir)
# dataloader = DataLoader(dataset, batch_size=32, num_workers=0,shuffle=True)

# Fetch the first item
# errorlist = 0

# dataset.__getitem__(0)

# for i in range(0,100000,5):
#     try:
#         dataset.__getitem__(i)
#     except:
#         errorlist +=1
# print(f"{errorlist}/{1000000/5}")
# embed, label = dataset.__getitem__(1000)
# print(f"Embedding shape: {embed.shape}, Label: {label}")

In [4]:
csv_file = 'src/transformed/data.csv'
npy_dir = 'src/embedded_ef/'

dataset = CLSdata(csv_file=csv_file, npy_dir=npy_dir)



for i in range(len(dataset)):
    try:
        dataset.__getitem__(i)
    except Exception as e:
        print(e)
        print("Index:",i)

# tens.shape

In [4]:
print(len(dataset))

2367606


In [6]:
dataset.__getitem__(2367607)

IndexError: list index out of range

**We Must split the data first**

In [19]:
#Function for simple bootstrap aggregating

def bootstrap_samples(labels, n_samples=10):
    labels_arr = np.array(labels)
    n_labels = len(labels_arr)
    
    #lists for samples & indices
    bootstrap_samples = []
    bootstrap_indices = []
    for _ in range(n_samples):

        #Sample with replacement from the labels array
        sample_indices = np.random.choice(n_labels, size=n_labels, replace=True)
        bootstrap_sample = labels_arr[sample_indices]
        
        bootstrap_samples.append(bootstrap_sample)
        bootstrap_indices.append(sample_indices) 
    
    return bootstrap_samples, bootstrap_indices

labels = np.array(dataset.labels[:-2])
indices = np.arange(len(labels))

train_idx, temp_idx, train_labels, temp_labels = train_test_split(
    indices, labels, test_size=0.4, stratify=labels, random_state=0)

samples = bootstrap_samples(train_labels)
samples
trainx,testx = samples[1][0],samples[1][1]
trainx
# trainsamples

array([1361132,   86821, 1045130, ...,  539069, 1046478, 1128966])

In [32]:
1420563 / 100

14205.63

In [21]:
#Basic model with no advanced implementations like dropout or batch norm
class Model1(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(768,200)
        self.layer2 = nn.Linear(200,500)
        self.layer3 = nn.Linear(500,2)

    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        return F.softmax(self.layer3(x),dim=0)
    

In [17]:
bert_model = BertModel.from_pretrained('bert-base-cased')


class BertClassifier(nn.Module):
    def __init__(self, bert_model):
        super(BertClassifier, self).__init__()
        self.bert = bert_model
        self.linear = nn.Linear(bert_model.config.hidden_size, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_ids, attention_mask=None):
        print("Forward Pass")
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_hidden_state = outputs.last_hidden_state[:, 0, :]  # Using the hidden state corresponding to [CLS] token
        linear_output = self.linear(cls_hidden_state)
        output = self.sigmoid(linear_output)
        return output
    


#Awful model. Don't use!!!
# model = BertClassifier(bert_model)
# model.to(device)

BertClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_af

10

In [41]:

# Assuming Model1 is defined as per your snippet

# Initialize the model
model = Model1().to(device)

# Define your dataset
# Here you should load your dataset and prepare it
# For this example, let's assume we have a DataLoader that provides the entire dataset in one batch
trainsubset = Subset(dataset,trainx)
testsubset = Subset(dataset,testx)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=14205)
# Define the loss function
criterion = nn.CrossEntropyLoss()

# Initialize the optimizer
optimizer = optim.SGD(model.parameters())

# Define the number of epochs
epochs = 10

# Training loop
for epoch in range(epochs):
    print(f"----- EPOCH {epoch} -----")
    epoch_loss = 0  # To accumulate loss over the epoch
    for idx, (inputs, labels) in enumerate(data_loader):
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        # Compute loss
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Accumulate loss
        epoch_loss += loss.item()
        print(f"\rBatch {idx}, Loss: {loss.item()}", end = "", flush = True)

    avg_epoch_loss = epoch_loss / len(data_loader)
    print(f"Epoch {epoch+1}, Avg Loss: {avg_epoch_loss}")

----- EPOCH 0 -----
Batch 166, Loss: 0.6931471824645996Epoch 1, Avg Loss: 0.6931472281495968
----- EPOCH 1 -----
Batch 166, Loss: 0.6931471824645996Epoch 2, Avg Loss: 0.6931472281495968
----- EPOCH 2 -----
Batch 166, Loss: 0.6931471824645996Epoch 3, Avg Loss: 0.6931472281495968
----- EPOCH 3 -----
Batch 20, Loss: 0.6931472420692444

KeyboardInterrupt: 

In [25]:
# num_epochs = 5  # Example number of epochs
# optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
# criterion = nn.BCELoss().to(device)

# # Assuming model, optimizer, and criterion definitions are correct and provided above
# # model = BertClassifier(bert_model).to(device) has been defined earlier

# num_epochs = 5

# for epoch in range(num_epochs):
#     print(f"------ Epoch {epoch} ------")
#     for idx, (inputs, labels) in enumerate(subset_dataloader):
#         # print("Batch", idx)
#         optimizer.zero_grad()
        
#         # Correctly move inputs and labels to the device
#         inputs, labels = inputs[:, :512].to(device), labels.to(device)
#         # inputs, labels = inputs.to(device), labels.to(device)

#         # Forward pass
#         outputs = model(inputs.long())

#         # Calculate loss
#         loss = criterion(outputs.squeeze(), labels.float())
#         print(f"Loss: {loss.item()}")
        
#         # Backward pass and optimize
#         loss.backward()
#         optimizer.step()


------ Epoch 0 ------
Forward Pass


IndexError: index out of range in self